In [ ]:
!pip install gplearn

     |████████████████████████████████| 41 kB 151 kB/s 


In [ ]:
from gplearn.genetic import SymbolicRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.utils.random import check_random_state
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import graphviz

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = "/content/drive/MyDrive/avocado?"

In [ ]:
train_norm = pd.read_csv(f"{path}/data/gri_train_normalized.txt", sep=" ", header = None, names=["g", "r", "i", "zred"])
sdss = pd.read_csv(f"{path}/data/sdss_photz.txt", sep=" ", header = None, names=["u", "g", "r", "i", "z", "zred"])
sdss_colors = pd.read_csv(f"{path}/data/sdss_colors.txt", sep=" ", header = None, names=["u-g", "g-r", "r-i", "i-z", "zred"])
sdss_photz_train_normalized = pd.read_csv(f"{path}/data/sdss_photz_train_normalized.txt", sep=" ", header = None, names=["u", "g", "r", "i", "z", "zred"])
sdss_color_train = pd.read_csv(f"{path}/data/sdss_color_train.txt", sep=" ", header = None, names=["u", "g", "r", "i", "z", "zred"])

In [ ]:
x_train = train_norm.drop(['zred'], axis=1)
y_train = train_norm['zred']

x_train.shape, y_train.shape

((2728, 3), (2728,))

In [ ]:
x_sdss = sdss.drop(['zred'], axis=1)
y_sdss = sdss['zred']

x_sdss.shape, y_sdss.shape

((4072, 5), (4072,))

In [ ]:
x_colors = sdss_colors.drop(['zred'], axis=1)
y_colors = sdss_colors['zred']

x_colors.shape, y_colors.shape

((4072, 4), (4072,))

In [ ]:
x_photz_train_n = sdss_photz_train_normalized.drop(['zred'], axis=1)
y_photz_train_n = sdss_photz_train_normalized['zred']

x_photz_train_n.shape, y_photz_train_n.shape

((2728, 5), (2728,))

In [ ]:
x_color_train = sdss_color_train.drop(['zred'], axis=1)
y_color_train = sdss_color_train['zred']

x_color_train.shape, y_color_train.shape

((2728, 5), (2728,))

#Using gplearn with **gri_train_normalized.txt** data



In [ ]:
gp_norm = SymbolicRegressor(random_state=0)
gp_norm.fit(x_train, y_train)

SymbolicRegressor(const_range=(-1.0, 1.0), feature_names=None,
                  function_set=('add', 'sub', 'mul', 'div'), generations=20,
                  init_depth=(2, 6), init_method='half and half',
                  low_memory=False, max_samples=1.0,
                  metric='mean absolute error', n_jobs=1, p_crossover=0.9,
                  p_hoist_mutation=0.01, p_point_mutation=0.01,
                  p_point_replace=0.05, p_subtree_mutation=0.01,
                  parsimony_coefficient=0.001, population_size=1000,
                  random_state=0, stopping_criteria=0.0, tournament_size=20,
                  verbose=0, warm_start=False)

In [ ]:
print(gp_norm._program)
score_gp = gp_norm.score(x_train, y_train)
print('R2:', score_gp)

mul(mul(0.671, 0.700), sub(sub(sub(X1, -0.938), mul(X2, 0.554)), 0.074))
R2: 0.8504845419034952


In [ ]:
converter = {
    'add': lambda x, y : x + y,
    'sub': lambda x, y : x - y,
    'mul': lambda x, y : x*y,
    'div': lambda x, y : x/y,
    'sqrt': lambda x : x**0.5,
    'log': lambda x : log(x),
    'abs': lambda x : abs(x),
    'neg': lambda x : -x,
    'inv': lambda x : 1/x,
    'max': lambda x, y : max(x, y),
    'min': lambda x, y : min(x, y),
    'sin': lambda x : sin(x),
    'cos': lambda x : cos(x),
    'pow': lambda x, y : x**y,
}

In [ ]:
from sympy import *
next_e = sympify(str(gp_norm._program), locals=converter)
next_e

0.4697*X1 - 0.2602138*X2 + 0.4058208

#Using gplearn with **sdss_photz.txt** data

In [ ]:
gp_sdss = SymbolicRegressor(function_set=('add', 'sub', 'mul', 'div', 
                                                   'sqrt', 'log', 'abs', 'neg', 
                                                   'inv', 'max', 'min', 'sin', 'cos', 'tan'), random_state=0)
gp_sdss.fit(x_sdss, y_sdss)

SymbolicRegressor(const_range=(-1.0, 1.0), feature_names=None,
                  function_set=('add', 'sub', 'mul', 'div', 'sqrt', 'log',
                                'abs', 'neg', 'inv', 'max', 'min', 'sin', 'cos',
                                'tan'),
                  generations=20, init_depth=(2, 6),
                  init_method='half and half', low_memory=False,
                  max_samples=1.0, metric='mean absolute error', n_jobs=1,
                  p_crossover=0.9, p_hoist_mutation=0.01, p_point_mutation=0.01,
                  p_point_replace=0.05, p_subtree_mutation=0.01,
                  parsimony_coefficient=0.001, population_size=1000,
                  random_state=0, stopping_criteria=0.0, tournament_size=20,
                  verbose=0, warm_start=False)

In [ ]:
print(gp_sdss._program)
score_gp_sdss = gp_sdss.score(x_sdss, y_sdss)

cos(sqrt(add(mul(0.426, X3), X3)))


In [ ]:
from sympy import *
next_sdss = sympify(str(gp_sdss._program), locals=converter)
next_sdss

cos(1.19415241908225*X3**0.5)

#Using gplearn with **sdss_colors.txt** data




In [ ]:
gp_colors = SymbolicRegressor(random_state=0)
gp_colors.fit(x_colors, y_colors)

SymbolicRegressor(const_range=(-1.0, 1.0), feature_names=None,
                  function_set=('add', 'sub', 'mul', 'div'), generations=20,
                  init_depth=(2, 6), init_method='half and half',
                  low_memory=False, max_samples=1.0,
                  metric='mean absolute error', n_jobs=1, p_crossover=0.9,
                  p_hoist_mutation=0.01, p_point_mutation=0.01,
                  p_point_replace=0.05, p_subtree_mutation=0.01,
                  parsimony_coefficient=0.001, population_size=1000,
                  random_state=0, stopping_criteria=0.0, tournament_size=20,
                  verbose=0, warm_start=False)

In [ ]:
print(gp_colors._program)
score_gp_c = gp_colors.score(x_colors, y_colors)
print('R2:', score_gp_c)

mul(sub(div(div(X3, X3), div(add(X1, X2), X3)), X2), -0.693)
R2: 0.5389975346578862


In [ ]:
next_colors = sympify(str(gp_colors._program), locals=converter)
next_colors

0.693*X2 - 0.693*X3/(X1 + X2)

#Using gplearn with **sdss_photz_train_normalized.txt** data

In [ ]:
gp_photz_train_n = SymbolicRegressor(random_state=0)
gp_photz_train_n.fit(x_photz_train_n, y_photz_train_n)

SymbolicRegressor(const_range=(-1.0, 1.0), feature_names=None,
                  function_set=('add', 'sub', 'mul', 'div'), generations=20,
                  init_depth=(2, 6), init_method='half and half',
                  low_memory=False, max_samples=1.0,
                  metric='mean absolute error', n_jobs=1, p_crossover=0.9,
                  p_hoist_mutation=0.01, p_point_mutation=0.01,
                  p_point_replace=0.05, p_subtree_mutation=0.01,
                  parsimony_coefficient=0.001, population_size=1000,
                  random_state=0, stopping_criteria=0.0, tournament_size=20,
                  verbose=0, warm_start=False)

In [ ]:
print(gp_photz_train_n._program)
score_gp_ptn = gp_photz_train_n.score(x_photz_train_n, y_photz_train_n)
print('R2:', score_gp_ptn)

sub(sub(X2, -0.247), sub(X2, mul(sub(sub(X2, -0.247), sub(X2, mul(0.090, sub(X2, mul(sub(sub(X2, -0.247), sub(X2, mul(0.090, X2))), sub(X2, -0.247)))))), sub(X2, -0.247))))
R2: 0.7894709241522226


In [ ]:
next_ptn = sympify(str(gp_photz_train_n._program), locals=converter)
next_ptn

(X2 + 0.247)*(0.09*X2 - 0.09*(0.09*X2 + 0.247)*(X2 + 0.247) + 0.247) + 0.247

#Using gplearn with **sdss_color_train.txt** data

In [ ]:
gp_color_train = SymbolicRegressor(random_state=0)
gp_color_train.fit(x_color_train, y_color_train)

SymbolicRegressor(const_range=(-1.0, 1.0), feature_names=None,
                  function_set=('add', 'sub', 'mul', 'div'), generations=20,
                  init_depth=(2, 6), init_method='half and half',
                  low_memory=False, max_samples=1.0,
                  metric='mean absolute error', n_jobs=1, p_crossover=0.9,
                  p_hoist_mutation=0.01, p_point_mutation=0.01,
                  p_point_replace=0.05, p_subtree_mutation=0.01,
                  parsimony_coefficient=0.001, population_size=1000,
                  random_state=0, stopping_criteria=0.0, tournament_size=20,
                  verbose=0, warm_start=False)

In [ ]:
print(gp_color_train._program)
score_gp_ct = gp_color_train.score(x_color_train, y_color_train)
print('R2:', score_gp_ct)

mul(sub(X3, X2), -0.560)
R2: 0.6212252870497356


In [ ]:
next_ct = sympify(str(gp_color_train._program), locals=converter)
next_ct

0.56*X2 - 0.56*X3